# **Project Name**    - Rosamann Retails Sales Prediction



##### **Project Type**    - Regression
##### **Contribution**    - Individual
##### **Member 1 -**  - Jayesh Prakash Dahiwale


# **Project Summary -**

**Rossmann operates over 3,000 drug stores in 7 european countries. So our porject aim is to predict their daily sales for upto six weeks in advance. Stores sales are influenced by many factors, including promotions, competitions,school and state holidays, seasonality and locality. So In this project we have been provided the data from 1,500 Rossmann stores. So our task is to forecast the "Sales" column for the test** 
##<p>Provided there are two datasets, one is **Store** dataset having <b>1,115</b> observations in it with <b>10</b> columns and It gives us static information about each store such as the model and assortment of the store, information about the nearest competitor store, and whether or not they participate in the consecutive promotion "Promo2". Largely we're looking at numerical and date data, but Store Type and Assortment are flagged with letters to indicate store models and assorment level, per the variable explanations, as well as the PromoInterval column listing abbreviated months.</p>
## <p>Other datatset is about **Sales** dataset having <b>1,017,209</b> observations in it with <b>9</b> columns and It gives us static information about each store such as the model and assortment of the store, information about the nearest competitor store, and whether or not they participate in the consecutive promotion "Promo2". Largely we're looking at numerical and date data, but Store Type and Assortment are flagged with letters to indicate store models and assorment level, per the variable explanations, as well as the PromoInterval column listing abbreviated months.</p>

# **GitHub Link -**

https://github.com/Jayeshdahiwale/RegressionCapstoneProjectII

# **DataSet Link -**


Dataset DriveLink:https://drive.google.com/drive/folders/1XtTzgmtM-e9jGHbIq8pLzIbo3C8NF1T2?usp=share_link



## **Questions for analysis are as follows**
**1) Visualising the distribution of "Sales" & "Customers"?**<br>
**2) Statistics of Sales column ?** <br>
**3) Which rows are unnecessary and need to be removed ??** <br>
**4) What are the outliers ?** <br>
**5) Establishing relationship between Sales and Customers ?** <br>
**6) How stores are performing in Sales by month based on Assortment type ?**<br>
**7) How UPT metric compares across stores of different assortment types ?**<br>
**8)Correlation between competition distance and UPT metric?**<br>
**9)Which linear regression model is best?**<br>




#### **Define Your Business Objective?**

Increase the number of sales by predicting the rates at at optimal rate and finding the best suitalbe condition which attract the customers thereby increasing the profit for the Drug Store.

## **Importing the required packages**

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import math

We'll begin with the required libraries and reading our .csv files

In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Reading the dataset file
# dataset file googgle drive link : https://drive.google.com/file/d/185htr6OyxAZ0xb9QwxHpsUn8eTlq58v5/view?usp=share_link
working_directory_path = '/content/drive/MyDrive/MachineLearningAlmabetterJourney/CapstoneProject2/'



In [ ]:
store_lookup = pd.read_csv(working_directory_path + 'store.csv')

In [ ]:
sales_data = pd.read_csv(working_directory_path + 'Rossmann Stores Data.csv')


In [ ]:
store_lookup.head()

In [ ]:
store_lookup.shape

In [ ]:
store_lookup.info()

In [ ]:
def null_cols(data):
  columns = data.columns
  null_cols = [(col,data[col].isna().sum()) for col in columns if data[col].isna().sum()!=0]
  return null_cols

In [ ]:
null_cols(store_lookup)

We can see in store lookup table there are total 6 null columns

In [ ]:
# Lets see the sales data
sales_data.head()

In [ ]:
sales_data.shape

In [ ]:
sales_data.info()

These is no null columns in this dataset

In [ ]:
#lets see the starting date and end date of sales data
print(sales_data['Date'].min())
print(sales_data['Date'].max())

**Now we will merge the two data resources so we can more easily work with them. We will join the tables baes on the shared store column, which is a foreign key in the sales_data table and primary key in the stores_llokup table, so we'll validate the merge baed on this many to one relationship**

In [ ]:
merged_sales = sales_data.merge(store_lookup,how='left',on='Store',validate = 'many_to_one')

In [ ]:
merged_sales.info()

In [ ]:
merged_sales.shape

In [ ]:
merged_sales.head()

In [ ]:
merged_sales.describe()

In [ ]:
merged_sales.info()

**Lets check the skewness of columns i.e.** <br>
 "Day of Week","Sales","Customers","Competion Distance"

In [ ]:
merged_sales[['DayOfWeek','Sales','Customers','CompetitionDistance']].skew()

we can see here competition distance,customers are positively skewed<br>


# **<p>Lets check the distribution bell peak using kurotsis</p>**

In [ ]:
merged_sales[['DayOfWeek','Sales','Customers','CompetitionDistance']].kurtosis()

We can see the that competition distance,and customers has highest peak...And sales and Day of week has negative kurtosis

# **Question 1: Visualising the distribution of Sales and Customer columns**

# **<p>Lets visualize the skewness and kurtosis</p>**

In [ ]:
fig,axes = plt.subplots(1,2)
sns.distplot(merged_sales['Sales'],color="y",ax=axes[0])
sns.distplot(merged_sales['Customers'],color="y",ax=axes[1])
fig.set_size_inches(15,5)

Histograms of our Sales and Customers values show us a positive skew and high kurtosis.

# **Question 2: Statistics of Sales column**

# <p>**Next we will take a closer look at our statistics for sales column.**</p>

In [ ]:
print("skew     " + str(round(merged_sales['Sales'].skew(),6)))
print("kurtosis " + str(round(merged_sales['Sales'].kurtosis(),6)))
print(merged_sales['Sales'].describe().round(3))
print("mode     " + str(merged_sales['Sales'].mode()))

We can see our sales figures have a slightly positive skew, with the **mean (5773.819)** only slightly larger than the **median (5744.000)**, suggesting most outliers are to the right of the mean.

High kurtosis indicates it's **leptokurtic** with the likelihood of heavy tails and outliers that may be extreme. Considering our min and max values of **0** and **41,551** sales, we aren't surprised to see there may be some extreme outliers.

The max value well above the mean of 5,773.819 and outside the **standard deviation of 3849.926** helps us see how our mean ends up getting pulled slightly to the right for our positive skew.

There is no mode as we don't have any stores recording the exact same number of sales on any days, which isn't surprising.

# Data Cleaning

#<p>Datetime</p>

First we'll convert our Date column to datetime for easier filtering later on.

With an .info() call we can confirm our Date column was successfully converted to datetime.

We'll also expand our Date column into separate Month, Day of Month, and Year columns for easier filtering.

In [ ]:
merged_sales['Date'] = pd.to_datetime(merged_sales['Date'], format="%Y-%m-%d", errors='raise')

In [ ]:
merged_sales.info()

In [ ]:
merged_sales["Year"] = merged_sales["Date"].dt.year
merged_sales["Month"] = merged_sales["Date"].dt.month
merged_sales["DayOfMonth"] = merged_sales["Date"].dt.day

# **Duplicates**

We'll do a quick check for and delete any rows that are complete duplicates of another row, as we should only have one entry for each store and date.

In [ ]:
len(merged_sales[merged_sales.duplicated()])

# **Text and Expected Value**

In [ ]:
merged_sales.loc[merged_sales['StateHoliday']==0,'StateHoliday'] = '0'

Next we'll check our four columns that contain strings to make sure we have no inappropriately entered data. Using str.strip() to remove any accidental leading or trailing spaces.

In [ ]:
for col in merged_sales:
  
  if merged_sales[col].dtype == object and col != 'StateHoliday':
    merged_sales[col] = merged_sales[col].str.strip()

Next we'll look at value_counts for those columns and confirm only expected values are found.

In [ ]:
for col in merged_sales:
  if merged_sales[col].dtype==object:
    print(merged_sales[col].value_counts().sort_index())

In [ ]:
merged_sales.info()

Everything is looking as expected in our columns that contain text, and because each column only has 3-4 unique values we can see we don't need to worry about changing anything to lower, upper, or proper case.

We'll also do a quick check of columns we expect to only have a few unique values or binary flags, such as DayOfWeek or Promo, to make sure there's nothing unexpected there.

In [ ]:
check_cols = [ 'Open', 'Promo', 'Promo2', 'SchoolHoliday',  'DayOfWeek', 'CompetitionOpenSinceMonth',  'CompetitionOpenSinceYear', 'Promo2SinceWeek', 'Promo2SinceYear']
for col in check_cols:
    print(col)
    print(sorted(merged_sales[col].unique()))

We see the relevant binary flags, day of weeks, week, month, and year numeric values we expect.

Because the CompetitionOpenSinceMonth, CompetitionOpenSinceYear, Promo2SinceWeek, and Promo2SinceYear columns are only using whole numbers and they are a discrete value, we will change them from floats to integers.

In [ ]:
merged_sales['CompetitionOpenSinceMonth'] = merged_sales['CompetitionOpenSinceMonth'].convert_dtypes()
merged_sales['CompetitionOpenSinceYear'] = merged_sales['CompetitionOpenSinceYear'].convert_dtypes()
merged_sales['Promo2SinceWeek'] = merged_sales['Promo2SinceWeek'].convert_dtypes()
merged_sales['Promo2SinceYear'] = merged_sales['Promo2SinceYear'].convert_dtypes()

In [ ]:
merged_sales.info()

Now we can see that CompetitionDistance, our only continuous value, is our only column with floats.

#  **Question3):Removing rows and columns**

Below I look at entries for stores on days they were closed.

For the purposes of our analysis I've chosen to drop these rows, as no sales are recorded on days stores are closed. The zero sales recorded for each of these rows lowers the average sales, and we can see this by comparing the mean Sales for all entries in our table to the mean Sales of only days that stores were open. If we filter for entries of stores that are closed we'll see a return of **172,817** rows, all of which record the expected 0 sales, lowering our mean Sales statistic.

**The potential information lost here is if we want to compare stores based on the number of days they are open or closed**, but that is beyond the scope of our analysis for now. To avoid losing this information we will make a copy of our dataframe with only the days stores are open, to further be referred to as sales, rather than altering merged_sales in case we wish to access this data at a later time.

In [ ]:
## mean sales including entries for days stores are closed
merged_sales['Sales'].mean()

In [ ]:
#Not lets check the mean only for the data when the store is open
merged_sales[merged_sales['Open']==1].Sales.mean()

In [ ]:
## confirming all entries where the store is  marked as closed have 0 sales
merged_sales.loc[merged_sales["Open"] == 0, ['Sales', 'Customers']].value_counts()


In [ ]:
## creating new sales dataframe with only entries for days stores are open
sales = merged_sales.drop(index=(merged_sales[merged_sales["Open"] == 0]).index, axis=1)

Now as we chose to delete any rows for days a store wasn't open, our Open column should only contain the value 1 now and is redundant, so we'll remove that

In [ ]:
sales.drop(columns=["Open"], inplace=True)

# **Question 4 : What are the outliers in the dataset ?**


Next we'll take a look at any outliers we may need to treat.

In [ ]:
sales.plot(y=['Sales', 'Customers', 'CompetitionDistance'], 
           kind='box', subplots=True, layout=(2,2), figsize=(15,15))

From the box plots above we can see that Sales, Customers, and CompetitionDistance all appear to have significant outliers, so we'll explore further by calculating and investigating the outliers for each one.


# **Sales Outliers**



We will start with the Sales column.

In [ ]:
def calculate_outlier(df,column): ## function for calculating outliers
    Q3 = df[column].quantile(0.75)
    Q1 = df[column].quantile(0.25)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    percent_outliers = round(((df[df[column] > upper].shape[0]) + (df[df[column] < lower].shape[0])) / df.shape[0] * 100, 2)
    return lower, upper, percent_outliers

In [ ]:
col = 'Sales'
lower_sales, upper_sales, percent_outliers_sales = calculate_outlier(sales, col)

print("lower band = " + str(lower_sales))
print("upper band = " + str(upper_sales))
print("percentage of sales that are outliers = " + str(percent_outliers_sales) + "%")
print('Number of observations in which sales leass than <0 are '+str(len(sales[sales.Sales <0])))

We know from our summary statistics that there aren't any sales below 0, so we'll just look at the upper outliers that we've calculated for the Sales column.

In [ ]:
sales[sales[col] > upper_sales]

While 30,769 is a lot of values, we can see from our calculte_outlier function that these outliers only account for 3.64% of all our sales values.

We'll look further to see if we see any trends with the outliers based on Month or Type of Store.

In [ ]:
sales_outliers_by_month = pd.pivot_table((sales.loc[sales[col] > upper_sales]), index='Month', values='Sales', aggfunc='count')

sales_outliers_by_month.plot(y='Sales', kind='bar', figsize=(10,5), title="# of Sales Outlier Entries by Month")
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

In [ ]:
sales_outliers_by_stype = pd.pivot_table((sales.loc[sales[col] > upper_sales]), index='StoreType', values='Sales', aggfunc='count')

sales_outliers_by_stype.plot(y='Sales', kind='bar', figsize=(6,6), 
                             title="# of Sales Outlier Entries by Store Type", 
                             color=['blue','red','green','orange'])
plt.ylabel('Sales')
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

When we look at the Sales outliers by month, we see the most represented month is **December at 17.33%**, which is unsurprising given the Christmas holidays. However, when we look at the outliers by Store Type we see that the **61.71% majority** are coming from **Type A stores**, while **Type B, C, D** are more equally represented at **11-15%**. This suggests that Type A stores may be the best performers in regards to outstanding sales days, and is worth looking into further.

Below we will treat our Sales outliers by imputing them with our upper range value we calculated earlier, **13611.5**, rounded up to **13612** as our Sales column is a measure of discrete values using whole numbers. As these outliers represent exceptionally high sales day, they are intended to be high numbers, but we would like to treat the outliers to limit their influence on any future modelling. As such imputing with our upper range value feels more appropriate than using our mean Sales value.

We also save this a new dataframe going forward, to further be referenced to as sales_treated, so that we can preserve our sales dataframe with the outliers intact, should we wish to investigate them further.

In [ ]:
sales_treated = sales.copy()

In [ ]:
sales_treated.loc[sales_treated[col] > upper_sales, 'Sales'] = 13612

Further check whether our imputation worked or not

In [ ]:
sales_treated[sales_treated['Sales'] > 13612]

# **Customer Outliers**

Next we'll calculate and investigate our Customer outliers.

In [ ]:
col = 'Customers'
lower_cust, upper_cust, percent_outliers_cust = calculate_outlier(sales_treated, col)

print(str(lower_cust) + ", " + str(upper_cust) +", " + str(percent_outliers_cust) + "%")
print('Number of rows when customer < 0 : ' + str(len(sales[sales_treated['Customers']<0])))

Similar to Sales, we know from our summary statistics that we don't have any Customer values below 0, so we'll just look at our upper range value.

In [ ]:
sales_treated[sales_treated['Customers'] > upper_cust]

We Can see right away that several of these entries have a **Sales value of 13,612**, which we know to be our newly imputed upper range value for Sales outliers. We expect a high correlation between Customers driving Sales, so we'll check to see how much crossover we have between our Sales and Customers outliers.\

In [ ]:
sales_treated[(sales_treated['Customers'] > upper_cust) & (sales_treated['Sales'] == 13612)]

We can see a crossover of **21,420 rows**, or approximately **52%** of our Customer outlier entries are also Sales outlier entries.

We will also investigate how these Customer outliers break down by Month and StoreType just as we did with our Sales outliers.

In [ ]:
cust_outliers_by_month = pd.pivot_table((sales_treated.loc[sales_treated[col] > upper_cust]), index='Month', values='Customers', aggfunc='count')

cust_outliers_by_month.plot(y='Customers', kind='bar', figsize=(10,5), title="# of Customer Outlier Entries by Month")
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

In [ ]:
cust_outliers_by_stype = pd.pivot_table((sales_treated.loc[sales_treated[col] > upper_cust]), index='StoreType', values='Customers', aggfunc='count')

cust_outliers_by_stype.plot(y='Customers', kind='bar', figsize=(6,6), 
                             title="# of Customer Outlier Entries by Store Type", 
                             color=['blue','red','green','orange'])
plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1.1))
plt.show()

**December** is our most represented month for Customer outliers, similar to our Sales outliers, but it's percentage of outliers is less than we saw with our Sales.

We also see **store Type A** with the strongest showing when we break down the outliers by store type. Much like the Sales outliers **Type A stores** represent a **strong 60%+** of the outliers. Surprisingly, **Type D stores** represent a tiny **1.02%** of these Customer outliers, where as they represented the second largest percentage of Sales outliers at **15.29%**. Further investigation into the number of items bought (Sales) per transaction (Customer) may prove insightful.

Similar to our Sales outliers, we will also limit our Customer outliers to our calculated upper range, by imputing them to **1,454,** so as to limit their influence but also indicate that they're meant to be high numbers.

In [ ]:
sales_treated.loc[sales_treated['Customers'] > upper_cust, 'Customers'] = 1454

In [ ]:
sales_treated.loc[sales_treated['Customers'] > upper_cust, 'Customers']

# **Missing Values**

Next we'll take a look at missing values. We'll start by assessing how many we have and where.

In [ ]:
sales_treated.isna().sum()

In [ ]:
(sales_treated.isna().sum() * 100 / sales_treated.shape[0]).round(2)   ## missing values as a % of all values in the column

While we see a rather high 50% missing values rate for Promo2SinceWeek and Promo2SinceYear we can check and see if these all just correspond to stores that aren't running Promo2, and hence would not be expected to have valid data for these columns.

In [ ]:
print(sales_treated[sales_treated['Promo2'] == 0].shape[0])
print(sales_treated['Promo2SinceWeek'].isna().sum())
print(sales_treated['Promo2SinceYear'].isna().sum())

As the number of entries with missing **Promo2SinceWeek** and **Promo2SinceYear** values corresponds with the number of entries where the store isn't participating in Promo2, we can rest assured that the lack of data here is appropriate.



The other area we see missing data is in relation to competition stores, mostly in the two columns relating when the nearest competitor store opened, and to much lesser extent in **CompetitionDistance**

The CompetitionDistance values may be used for analyzing how stores perform based on how close their competition is, and as such missing values could skew such analysis. 0 would be an inappropriate replacement for these null values as it would indicate that the competition stores are incredibly close. As such we'll look to replace these missing values with the mean CompetitionDistance based on the StoreType.

In [ ]:
mean_type_a = round(store_lookup.loc[(store_lookup['StoreType'] == 'a'), 'CompetitionDistance'].mean(), 1)
mean_type_b = round(store_lookup.loc[(store_lookup['StoreType'] == 'b'), 'CompetitionDistance'].mean(), 1)
mean_type_c = round(store_lookup.loc[(store_lookup['StoreType'] == 'c'), 'CompetitionDistance'].mean(), 1)
mean_type_d = round(store_lookup.loc[(store_lookup['StoreType'] == 'd'), 'CompetitionDistance'].mean(), 1)

print("The mean Compeition Distance for stores of type A is " + str(mean_type_a))
print("The mean Compeition Distance for stores of type B is " + str(mean_type_b))
print("The mean Compeition Distance for stores of type C is " + str(mean_type_c))
print("The mean Compeition Distance for stores of type D is " + str(mean_type_d))

In [ ]:
sales_treated.loc[sales_treated['StoreType'] == 'a'] = sales_treated.loc[sales_treated['StoreType'] == 'a'].fillna(value={"CompetitionDistance" : mean_type_a})
sales_treated.loc[sales_treated['StoreType'] == 'b'] = sales_treated.loc[sales_treated['StoreType'] == 'b'].fillna(value={"CompetitionDistance" : mean_type_b}) 
sales_treated.loc[sales_treated['StoreType'] == 'c'] = sales_treated.loc[sales_treated['StoreType'] == 'c'].fillna(value={"CompetitionDistance" : mean_type_c}) 
sales_treated.loc[sales_treated['StoreType'] == 'd'] = sales_treated.loc[sales_treated['StoreType'] == 'd'].fillna(value={"CompetitionDistance" : mean_type_d}) 

In [ ]:
sales_treated.isna().sum()

Next we will replace the null values in our PromoInterval column. As this column contains strings listing the months the Promo2 starts anew, we will replace the missing values with a string 'NA' for Not Applicable.

In [ ]:
sales_treated.loc[sales_treated['PromoInterval'].isna(), 'PromoInterval'] = "NA"

In [ ]:
sales_treated['PromoInterval'].value_counts()

For the missing values in **'CompetitionOpenSinceYear'** & **'Promo2SinceYear'** it would be inappropriate to replace them with zeros, as further calculations with the years in that column could lead to values suggesting a competition store has been open **2022** years. As such, we'll impute these missing values with the current year, so if calculations are done to find how long the nearest competition store has been open since these instances of no competition store nearby will return zero years.

In the same manner we will impute the missing values in **'CompetitionOpenSinceMonth'** with the **current month**, and 'Promo2SinceWeek' with the current week.

In [ ]:
sales_treated.loc[sales_treated['CompetitionOpenSinceYear'].isna(), 'CompetitionOpenSinceYear'] = dt.datetime.now().year

sales_treated.loc[sales_treated['Promo2SinceYear'].isna(), 'Promo2SinceYear'] = dt.datetime.now().year

sales_treated.loc[sales_treated['CompetitionOpenSinceMonth'].isna(), 'CompetitionOpenSinceMonth'] = dt.datetime.now().month

sales_treated.loc[sales_treated['Promo2SinceWeek'].isna(), 'Promo2SinceWeek'] = dt.datetime.now().isocalendar()[1]

In [ ]:
# lets see the null counts
sales_treated.isna().sum()

# **Question 5: Establising relationship between Sales and Customers ?**

 **Adding an UPT column**

Based on our investigation of Sales and Costumers outliers, particularly how they're represented by different store types, I thought it would be worthwhile to add a calculated column to give us some insight into the relationship between Customers and Sales for each store and day.

In [ ]:
(sales_treated['Customers'] >= sales_treated['Sales']).value_counts()

We can see that for almost all entries we have in our dataframe, the number of Sales at a given store is greater than the number of Customers. Suggesting that the Customers value is derived by how many transactions there are at a store, and the Sales value is indicative of how many individual items are sold. Thus we can calculate the average number of items sold for each transaction as Units Per Transaction (UPT).

First we'll quickly investigate the 54 rows where there aren't more Sales than Customers.

In [ ]:
sales_treated[sales_treated['Customers'] >= sales_treated['Sales']]

We can see the majority of these days are entries with both zero Sales and zero Customers recorded. This seems odd for a day that the store is open. A quick check of merged_sales, which still has the Open column, gives us the same results and assures us that the stores are indeed marked as open on these days.

In [ ]:
merged_sales[(merged_sales['Customers'] >= merged_sales['Sales']) & merged_sales['Open'] == 1]

I can only speculate as to why we have entries for stores that are open but aren't recording any sales, perhaps a stocktake day? Let's also look at the cases that aren't zero Sales and zero Customers.

In [ ]:
sales_treated[sales_treated['Customers'] > sales_treated['Sales']]

I'm unsure of why we have two days with zero Sales and a small handful of Customers. As I have no explanation for these 2 days, nor the 52 other open days with zero Sales and Customers, I don't feel comfortable deleting them.

This does pose a small problem for calculating our average UPT, however. As such we will create our UPT column by dividing the day's Sales by the days Customers to find the average Units Per Transaction for each day and store. The resulting 52 null values will be imputed with a zero to reflect the zero Sales for those entries.

In [ ]:
sales_treated['UPT'] = sales_treated['Sales'] / sales_treated['Customers']

In [ ]:
sales_treated['UPT'].isna().sum()

In [ ]:
sales_treated.loc[sales_treated['UPT'].isna(), 'UPT'] = 0

In [ ]:
sales_treated['UPT'].isna().sum()

We'll also add this UPT column to our sales table which still contains our Sales and Customers outliers, so we can explore the UPT with the outliers as well.

In [ ]:
sales['UPT'] = sales['Sales'] / sales['Customers']
sales.loc[sales['UPT'].isna(), 'UPT'] = 0

# **Review of Summary Statistics Post Cleaning**

Let's quickly review some of our summary statistics now that we've cleaned our data a bit. We can look at the summary statistics for sales to see them as they were before treating outliers and imputing for missing data, and compare them to the sales_treated summary statistics to see how they've changed.

In [ ]:
merged_sales[['Sales', 'Customers', 'CompetitionDistance']].describe() ## BEFORE cleaning


In [ ]:
sales_treated[['Sales', 'Customers', 'CompetitionDistance', 'UPT']].describe() ## AFTER cleaning

We can see that the cleaning has narrowed our standard deviation for Sales and Customers, as well as raising the mean after removing the entries for closed stores.

In [ ]:
merged_sales[['Sales', 'Customers', 'CompetitionDistance']].skew() ## BEFORE cleaning

In [ ]:
sales_treated[['Sales', 'Customers', 'CompetitionDistance', 'UPT']].skew() ## AFTER cleaning

In [ ]:
merged_sales[['Sales', 'Customers', 'CompetitionDistance']].kurtosis() ## BEFORE cleaning

In [ ]:
sales_treated[['Sales', 'Customers', 'CompetitionDistance', 'UPT']].kurtosis() ## AFTER cleaning

In [ ]:
fig,axes = plt.subplots(1,2)
sns.distplot(sales_treated['Sales'],color="y",ax=axes[0])
sns.distplot(sales_treated['Customers'],color="y",ax=axes[1])
fig.set_size_inches(15,5)

Histograms of our Sales and Customers values shows us a slight positive skew, which changed a little after our cleaning, and we see a more significant change in kurtosis being lowered. We also see the effect of imputing our outliers with our upper range limit on the right side of either histogram.

 we now got results that are looking far closer to a standard normalization.

## **Exploratory Insights**

## **Question 6: How stores perform in Sales by month based of Assortment type?**

Let's explore how stores perform in Sales by month, based on Assortment type. We know that Assortment type A offers a "basic" assortment of merchandise, Type B offers and "extra" assortment, and type C offers an "extended" assortment.

Because our data ranges from Jan. 1, 2013 - July 31, 2015, we will exclude the 2015 data for now so as we are only looking at a complete years' worth of numbers.

In [ ]:
assortment_pivot_total_sales = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), ## Excluding 2015 data
               index='Month', values='Sales', columns='Assortment', aggfunc=np.sum)
assortment_pivot_total_sales

In [ ]:
assortment_pivot_total_sales.plot(kind='line', title='Total Sales by Month and Store Assortment', figsize=(7,8), grid=True)
plt.show()

A quick look at Sales by volume of total sales shows that stores of Assortment types A and C have significantly more volume than type B stores. Type B stores stay fairly consistent in total Sales volume across all months, with minor upticks during mid-year and end year. Type A and C stores can be seen to follow very similar trends in terms of Sales volume.

A quick look at how many stores we have of each Assortment type will show us significantly less stores of Assortment type B, which accounts for the significantly lower volume of Sales.

In [ ]:
pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), index='Assortment', values='Store', aggfunc='count')

Due to the vast differences in total Sales volume based on Assortment type, lets also look at the average number of Sales.

(Note: We could include our 2015 data since we're calculating the mean Sales now, but for the sake of consistency when comparing it with the total Sales we will continue to use the same 2013-2014 data.)

In [ ]:
assortment_pivot_avg_sales = pd.pivot_table((sales_treated[sales_treated['Year'] < 2015]), index='Month', values='Sales', columns='Assortment', aggfunc=np.mean)
assortment_pivot_avg_sales

In [ ]:
assortment_pivot_avg_sales.plot(kind='line', title='Average Sales by Month and Store Assortment', figsize=(7,5), grid=True)
plt.show()

Look at average Sales by store Assortment type we can see that stores of type B actually perform quite well when compared to types A and C, despite there being significantly less type B stores! Types A and C continue to follow very similar trends for Sales, but Type C stores consistently outperform type A stores.

